<a href="https://colab.research.google.com/github/ayushipradhan30/AIML-mini-projects/blob/main/Neural_Network_and_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout, Input
from keras import optimizers
from keras.optimizers import SGD, Adam

print('TensorFlow Version: ', tf.__version__)
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls '/content/drive/My Drive/train'

In [ ]:
file_path = '/content/drive/My Drive/train/SVHN_single_grey1.h5'
hdf = h5py.File(file_path, 'r')
hdf.keys()

In [ ]:
x_train = hdf['X_train']
x_test = hdf['X_test']
x_val = hdf['X_val']
y_train = hdf['y_train']
y_test = hdf['y_test']
y_val = hdf['y_val']

In [ ]:
print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)
print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)

In [ ]:
fig = plt.figure(figsize = (10,10))
cols = 10
rows = 10
for i in range(1, cols * rows + 1):
  img = x_test[i]
  fig.add_subplot(rows, cols, i)
  plt.imshow(img, cmap = 'gray')
plt.show()

In [ ]:
plt.imshow(x_train[0], cmap = 'gray')
plt.show()
print("label: ", y_train[0])

In [ ]:
plt.imshow(x_test[0], cmap = 'gray')
plt.show()
print("label: ", y_test[0])

In [ ]:
plt.figure(figsize = (10,1))
for i in range(10):
  plt.subplot(1, 10, i + 1)
  plt.imshow(x_train[i].reshape(32,32), cmap = 'gray', )
  plt.axis('off')
plt.show()
print('Labels for each of the above image: %s' %(y_train[0:10]))

## Implementing an optimal K-Nearest Neighbour Classifier

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

In [ ]:
x_train = np.asarray(x_train).reshape(42000, 1024)
x_test = np.asarray(x_test).reshape(18000, 1024)
x_val = np.asarray(x_val).reshape(60000, 1024)

In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_val = x_val.astype('float32') / 255

In [ ]:
neighbors = list(range(1, 30, 2))

In [ ]:
acc_score = []

for i in neighbors:
  knn = KNeighborsClassifier()
  knn.fit(x_train, y_train)
  y_pred = knn.predict(x_test)
  scores = accuracy_score(y_test, y_pred)
  acc_score.append(scores)

MSE = [1 - x for x in acc_score]

optimal_k = neighbors[MSE.index(min(MSE))]
print(optimal_k)

In [ ]:
knn = KNeighborsClassifier(n_jobs = -1, n_neighbors=optimal_k)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

print('\nClassification Metric Report:\n', classification_report(y_test, y_pred))
print('\nConfusion Matrix:\n', confusion_matrix(y_test, y_pred))
print("\nAccuracy Score: \n", accuracy_score(y_test, y_pred))

In [ ]:
plt.figure(figsize = (4,4))
plt.imshow(x_test[70].reshape(32,32))
plt.show()
image = x_test[70]
print("Label: \n", knn.predict(image.reshape(1, -1)))

Implementing Deep Neural Network

In [ ]:
x_train = np.asarray(x_train).reshape(42000, 1024)
x_test = np.asarray(x_test).reshape(18000, 1024)
x_val = np.asarray(x_val).reshape(60000, 1024)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=10)

In [ ]:
print("y_train:\n", y_train[0:5])
print("y_test:\n", y_test[0:5])
print("y_val:\n", y_val[0:5])

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

Implementing Neural Network

In [ ]:
#Initialize the Artificial Neural Network Classifier
keras_models = Sequential()

In [ ]:
#Input Layer with 512 nodes
keras_models.add(Dense (512, kernel_initializer='he_normal', input_shape = (1024,)))
#Activation Function
keras_models.add(Activation('relu'))

#Hidden Layer 1
keras_models.add(Dense(256, kernel_initializer='he_normal'))
keras_models.add(Activation('relu'))

#Hidden Layer 2
keras_models.add(Dense(128, kernel_initializer='he_normal'))
keras_models.add(Activation('relu'))

#Hidden Layer 3
keras_models.add(Dense(64, kernel_initializer='he_normal'))
keras_models.add(Activation('relu'))

#Hidden Layer 4
keras_models.add(Dense(32, kernel_initializer='he_normal'))
keras_models.add(Activation('relu'))

#Output Layer with 10 nodes
keras_models.add(Dense(10))
#Here, we are using softmax function because we have multiclass classsification
keras_models.add(Activation('softmax'))

In [ ]:
keras_models.summary()

In [ ]:
keras_models.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = keras_models.fit(x_train, y_train, validation_data = (x_val, y_val), batch_size = 200, epochs = 20, verbose = 1)

In [ ]:
results = keras_models.evaluate(x_val, y_val)
print(results[1])

In [ ]:
keras_models.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history_1 = keras_models.fit(x_train, y_train, validation_data = (x_val, y_val), batch_size = 200, epochs = 20, verbose = 1)

In [ ]:
results = keras_models.evaluate(x_val, y_val)
print('Validation accuracy using basic Neural Network (ADAM) : ', results[1])

In [ ]:
SGD = optimizers.SGD(learning_rate = 0.0001)
keras_models.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history_2 = keras_models.fit(x_train, y_train, validation_data = (x_val, y_val), batch_size = 200, epochs = 100, verbose = 1)

In [ ]:
results_sgd = keras_models.evaluate(x_val, y_val)
print('Validation accuracy using learning rate = 0.0001 (SGD) : ', results_sgd[1])

In [ ]:
results_on_val = pd.DataFrame({'Method':['Basic NN (SGD)'], 'Accuracy': round((results_sgd[1] * 100), 2)}, index=['1'])
results_on_val = results_on_val[['Method', 'Accuracy']]
results_on_val

In [ ]:
adam = optimizers.Adam(learning_rate = 0.0001)
keras_models.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['Accuracy'])
history_3 = keras_models.fit(x_train, y_train, validation_data = (x_val, y_val), batch_size = 200, epochs = 100, verbose = 1)

In [ ]:
results_adam = keras_models.evaluate(x_val, y_val)
print('Validation accuracy using learning rate = 0.0001 (Adam) : ', results_adam[1])

In [ ]:
temp_df = pd.DataFrame({'Method': ['Basic NN (Adam)'], 'Accuracy': round(results_adam[1] * 100, 2)}, index=[2])
results_on_val = pd.concat([results_on_val, temp_df])
result_on_val = results_on_val[['Method', 'Accuracy']]
result_on_val

In [ ]:
keras_models_2 = Sequential()

In [ ]:
#Input Layer with 512 nodes
keras_models_2.add(Dense (512, kernel_initializer='he_normal', input_shape = (1024,)))
#Activation Function
keras_models_2.add(Activation('relu'))

#Hidden Layer 1
keras_models_2.add(Dense(256, kernel_initializer='he_normal'))
keras_models_2.add(Activation('relu'))

#Hidden Layer 2
keras_models_2.add(Dense(128, kernel_initializer='he_normal'))
keras_models_2.add(Activation('relu'))

#Hidden Layer 3
keras_models_2.add(Dense(64, kernel_initializer='he_normal'))
keras_models_2.add(Activation('relu'))

#Hidden Layer 4
keras_models_2.add(Dense(32, kernel_initializer='he_normal'))
keras_models_2.add(Activation('relu'))

#Output Layer with 10 nodes
keras_models_2.add(Dense(10))
#Here, we are using softmax function because we have multiclass classsification
keras_models_2.add(Activation('softmax'))

In [ ]:
keras_models_2.summary()

In [ ]:
SGD = optimizers.SGD(learning_rate = 0.001)
keras_models_2.compile(optimizer = SGD, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history_2 = keras_models.fit(x_train, y_train, validation_data = (x_val, y_val), batch_size = 200, epochs = 100, verbose = 1)

In [ ]:
results_2 = keras_models_2.evaluate(x_val, y_val)
print('Validation accuracy using learning rate = 0.0001 (SGD) : ', results_2[1])

In [ ]:
temp_df = pd.DataFrame({'Method': ['Basic NN (SGD)'], 'Accuracy': round(results_2[1] * 100, 2)}, index=[3])
results_on_val = pd.concat([results_on_val, temp_df])
result_on_val = results_on_val[['Method', 'Accuracy']]
result_on_val

## Batch Normalisation

In [ ]:
keras_models_3 = Sequential()

In [ ]:
#Input Layer with 512 nodes
keras_models_3.add(Dense (512, kernel_initializer='he_normal', input_shape = (1024,)))
#Activation Function
keras_models_3.add(Activation('relu'))

#Hidden Layer 1
keras_models_3.add(Dense(256, kernel_initializer='he_normal'))
keras_models_3.add(Activation('relu'))

#Hidden Layer 2
keras_models_3.add(Dense(128, kernel_initializer='he_normal'))
keras_models_3.add(Activation('relu'))

#Hidden Layer 3
keras_models_3.add(Dense(64, kernel_initializer='he_normal'))
keras_models_3.add(Activation('relu'))

#Hidden Layer 4
keras_models_3.add(Dense(32, kernel_initializer='he_normal'))
keras_models_3.add(Activation('relu'))

#Output Layer with 10 nodes
keras_models_3.add(Dense(10))
#Here, we are using softmax function because we have multiclass classsification
keras_models_3.add(Activation('softmax'))